# House-Prices

## Importação das Bibliotecas

In [ ]:
import numpy as np # álgebra linear
import pandas as pd # processamento de dados

## Carregamento das bases de dados

In [ ]:
train = pd.read_csv('train.csv') 
test  = pd.read_csv('test.csv')

In [ ]:
train.shape

# Pre-processamento

In [ ]:
# Remove qualquer linha que tenha o valor SalePrice nulo
Target = 'SalePrice'
train.dropna(axis=0, subset=[Target], inplace=True)

In [ ]:
train.shape

In [ ]:
# Junta os DataFrames de treino e teste em apenas um, para ganhar consistência.
data=pd.concat([train.iloc[:,:-1],test],axis=0)

print('train df possui {} linhas e {} colunas'.format(train.shape[0],train.shape[1]))
print('test df possui {} linhas e {} colunas'.format(test.shape[0],test.shape[1]))
print('data df possui {} linhas e {} colunas'.format(data.shape[0],data.shape[1]))

In [ ]:
data = data.drop(columns=['Id'],axis=1)

## Valores Nulos

In [ ]:
# Exibe os valores faltantes e a porcentagem com relação ao total
def missingValuesInfo(df):
    total = df.isnull().sum().sort_values(ascending = False)
    percent = round(df.isnull().sum().sort_values(ascending = False)/len(df)*100, 2)
    temp = pd.concat([total, percent], axis = 1,keys= ['Total', 'Percent'])
    return temp.loc[(temp['Total'] > 0)]

missingValuesInfo(train)

In [ ]:
# Manipulação dos valores faltantes

def HandleMissingValues(df):
    # para colunas do tipo Object preenche com 'UNKNOWN'
    # para colunas do tipo Numeric preenche com a média
    
    # para cada coluna em df.columns (nomeadas como 'cname'), seleciona-se aquelas que são de tipo numérico
    num_cols = [cname for cname in df.columns if df[cname].dtype in ['int64', 'float64']]
    
    # para cada coluna em df.columns (nomeadas como 'cname'), seleciona-se aquelas que são de tipo objeto(colunas categóricas)
    cat_cols = [cname for cname in df.columns if df[cname].dtype == "object"]
   
    # Monta-se um array para receber os valores substitutos
    values = {}
    
    # colunas do tipo object recebem 'UNKNOWN'
    for a in cat_cols:
        values[a] = 'UNKNOWN'
    
     # colunas do tipo numérico recebem a média
    for a in num_cols:
        values[a] = df[a].median()
    
    # substitui todos os valores nulos do dataframe pelos contidos no array 'values'
    df.fillna(value=values,inplace=True)
    
HandleMissingValues(data)
data.head()

In [ ]:
data.Alley.unique()

In [ ]:
# Conferindo os valores nulos:
data.isnull().sum().sum()

## Transformação dos valores categóricos em numéricos

In [ ]:
# Categorical Feature Encoding: transformação dos valores categóricos em valores numéricos
# Utiliza-se One-Hot-Encoding

# Retorna as colunas do tipo 'object'
def getObjectColumnsList(df):
    return [cname for cname in df.columns if df[cname].dtype == "object"]

# Executa o One-Hot-Encoding
def PerformOneHotEncoding(df,columnsToEncode):
    return pd.get_dummies(df,columns = columnsToEncode)

cat_cols = getObjectColumnsList(data)
data = PerformOneHotEncoding(data,cat_cols)
data.head()

In [ ]:
data.shape

In [ ]:
# re-divide o dataframe 'data' em 'train' e 'test', uma vez que os valores estão pre-processados
train_data=data.iloc[:1460,:] # Pega do começo até a linha 1460
test_data=data.iloc[1460:,:] # Pega da linha 1460 (não incluída) até o final
print(train_data.shape)
test_data.shape

# Dados estão Transformados!!!

# Mineração

In [ ]:
# Get X,y for modelling
X=train_data
y=train.loc[:,'SalePrice']

## Utilização do RidgeCV

In [ ]:
from sklearn.linear_model import RidgeCV # Regressão de Ridge com validação cruzada incorporada

# Configura, realiza o fit e as predições no dataframe 'test'
ridge_cv = RidgeCV(alphas=(0.01, 0.05, 0.1, 0.3, 1, 3, 5, 10))
ridge_cv.fit(X, y)
ridge_cv_preds=ridge_cv.predict(test_data)

## Utilização do xgboost

In [ ]:
import xgboost as xgb # Extreme Gradient Boosting

# Configura, realiza o fit e as predições no dataframe 'test'
model_xgb = xgb.XGBRegressor(n_estimators=340, max_depth=2, learning_rate=0.2)
model_xgb.fit(X, y)
xgb_preds=model_xgb.predict(test_data)

In [ ]:
# O python é tão foda que quando vc escreve o que está aqui embaixo,
# ele vê que os dois arrays (ridge_cv_preds, xgb_preds) tem o mesmo tamanho,
# assim soma o valores de mesma posição e divide por 2, 
# ou seja, o resultado é a média das predições.
predictions = ( ridge_cv_preds + xgb_preds )/2

In [ ]:
# criar o dataframe para submissão
submission = {
    'Id': test.Id.values,
    'SalePrice': predictions
}
solution = pd.DataFrame(submission)
solution.head()

In [ ]:
#salva para o arquivo csv
solution.to_csv('submission.csv',index=False)

## Regressão Linear

In [ ]:
from sklearn import linear_model

# Configura, realiza o fit e as predições no dataframe 'test'
lr = linear_model.LinearRegression()
model = lr.fit(X, y)
linear_pred = model.predict(test_data)

In [ ]:
# criar o dataframe para submissão (Apenas linear)
submission = {
    'Id': test.Id.values,
    'SalePrice': linear_pred
}
solution = pd.DataFrame(submission)
#salva para o arquivo csv
solution.to_csv('submissionLinear.csv',index=False)

In [ ]:
# criar o dataframe para submissão (Apenas XGB)
submission = {
    'Id': test.Id.values,
    'SalePrice': xgb_preds
}
solution = pd.DataFrame(submission)
#salva para o arquivo csv
solution.to_csv('submissionXgb.csv',index=False)

# Verificando os resultados

In [ ]:
print ('RIDGE_CV RMSE: \n', ridge_cv.score(X, y))
print ('XGB RMSE: \n', model_xgb.score(X, y))
print ('Linear RMSE: \n', model.score(X, y))